In [1]:
import numpy as np

# numerical_flux.py

(2, 3, 3)
[[[1 1]
  [2 2]
  [1 1]]

 [[2 2]
  [4 4]
  [2 2]]

 [[1 1]
  [2 2]
  [1 1]]]
(2, 3, 3)
[[[1 2 1]
  [2 4 2]
  [1 2 1]]

 [[1 2 1]
  [2 4 2]
  [1 2 1]]]


In [181]:
A = np.arange(25).reshape(5,5)
A = A[...]
B = np.stack((A,A,A), axis=0)
print(B.shape)
# print(B[:,:,0])
# B = np.transpose(B, (0,1,2))

axes = [1,0,2]
B = np.transpose(B, np.argsort(axes))
# print(B)
# print(B[:,:,1:-1])

# axes = [2,1,0]
B = np.transpose(B, np.argsort(axes))
# print(B)
# print(B[:,:,1:-1])

B = np.moveaxis(B, 1, 2)
# print(B)

B = np.moveaxis(B, 1, 2)
# print(B)
# print(B.ndim)
# B = np.moveaxis(B,0,-1)
print(B)

# print(B.reshape(-1))

cnt, cntx, cnty, cntz = 0, 0, 0, 0
nsnc = B.reshape(-1).shape[0]

iczn, icyn, icxn = B.shape

for idx in range(nsnc):
    
#     print("cnt=%i, cntx=%i, cnty=%i, cntz=%i, cnty_r=%i" %(cnt, cntx, cnty, cntz, cnty % (icxn * icyn)))
    
    cntx += 1
    if cntx % icxn == 0:
        cnty += 1
        cntx = 0
        
        if cnty % icyn == 0:
            cntz += 1
            cnty = 0
            
    cnt += 1

(3, 5, 5)
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]
  [20 21 22 23 24]]

 [[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]
  [20 21 22 23 24]]

 [[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]
  [20 21 22 23 24]]]


In [312]:
import numba as nb
from scipy import signal

@nb.jit(nopython=True, cache=True)
def dict_func(lst,a):
    plst = lst[1:-1,1:-1,:]
    leftx = plst[:,:,:-1]
    rightx = plst[:,:,1:]

    x_fluxes = (rightx - leftx)
#     print(x_fluxes.shape)

    plst = lst[1:-1,:,1:-1]
    lefty = plst[:,:-1,:]
    righty = plst[:,1:,:]

    y_fluxes = (righty - lefty)
#     print(y_fluxes.shape)

    plst = lst[:,1:-1,1:-1]
    leftz = plst[:-1,:,:]
    rightz = plst[1:,:,:]

    z_fluxes = (rightz - leftz)
#     print(z_fluxes.shape)
    p = - x_fluxes[:,:,:-1] + x_fluxes[:,:,1:] \
        - y_fluxes[:,:-1,:] + y_fluxes[:,1:,:] - z_fluxes[:-1,:,:] + z_fluxes[1:,:,:]
#     lst = lst / 16
    return p
    
# np.random.seed(555)
shp = lst.shape

# # p = np.zeros_like(lst)
# plst = lst.tolist()
# plst = lst[:,:,-1][...,None] + lst[:,:,:] + lst[:,:,0][...,None]
lst = np.random.randint(15,size=(64,35,64))
lst = np.pad(lst,((0,0),(0,0),(1,1)),mode='wrap')
lst = np.pad(lst,((0,0),(1,1),(0,0)),mode='wrap')
lst = np.pad(lst,((1,1),(0,0),(0,0)),mode='constant')

for i in range(200):
    p = dict_func(lst,a)

print(p.shape)

(64, 35, 64)


In [327]:
lst = np.random.randint(15,size=(10,20,30))

@nb.jit(nopython=True, cache=True)
def tpose(lst):
    lst = np.moveaxis(lst,0,-1)
    print(lst.shape)

# tpose(lst)
lst0 = lst.reshape(20,30,10)
lst = np.moveaxis(lst,0,-1)


grid = np.arange(27).reshape(3,3,3)
print(grid.shape)

(20, 30, 10)
(20, 30, 10)
False
(3, 3, 3)


In [417]:
sz = 25

lst = np.random.randint(15,size=(64,35,64))
bx,by,bz = lst.shape[0],lst.shape[1],lst.shape[2]
print(bx,by,bz)

def op(v):
    v = v.reshape(bx,by)
    v = np.pad(v[2:-2,2:-2],(2,2),mode='constant')
    lap = np.diag(np.ones(v.shape[0]))
    lap = np.dot(lap,v)

    return lap

def lap3D(p):
    shp = np.zeros(3, dtype='int')
    shp[0], shp[1], shp[2] = 64, 35, 64
    print(shp)
    p = p.reshape(shp[0],shp[1],shp[2])
    print(p.shape)
    coeff = 1./16
    hplusx, hplusy, hplusz = 1.0, 1.0, 1.0
    hcenter = 1.0
    oodx2, oody2, oodz2 = 1.0, 1.0, 1.0
    lap = np.zeros_like(p)
    
    pinner = p[1:-1,1:-1,:]
    leftx = pinner[:,:,:-1]
    rightx = pinner[:,:,1:]

    x_fluxes = (rightx - leftx)
#     print(x_fluxes.shape)

    pinner = p[1:-1,:,1:-1]
    lefty = pinner[:,:-1,:]
    righty = pinner[:,1:,:]

    y_fluxes = (righty - lefty)
#     print(y_fluxes.shape)

    pinner = p[:,1:-1,1:-1]
    leftz = pinner[:-1,:,:]
    rightz = pinner[1:,:,:]

    z_fluxes = (rightz - leftz)
#     print(z_fluxes.shape)
    lap[1:-1,1:-1,1:-1] = hplusx * oodx2 * coeff * (- x_fluxes[:,:,:-1] + x_fluxes[:,:,1:]) \
        + hplusy * oody2 * coeff * (- y_fluxes[:,:-1,:] + y_fluxes[:,1:,:]) \
        + hplusz * oodz2 * coeff * (- z_fluxes[:-1,:,:] + z_fluxes[1:,:,:]) \
        + hcenter * p[1:-1,1:-1,1:-1]
    lap = lap.reshape(-1)
    return lap

from scipy.sparse import linalg as la

b = np.arange(sz).reshape(5,5)
b = np.pad(b,(2,2),mode='wrap')

b = b.reshape(-1)


def wrap(xk):
    xk = xk.reshape(bx,by)
    return xk

p = lst.reshape(-1)
pshp = p.shape[0]
print(p.shape)
D = la.LinearOperator((pshp,pshp),lap3D)
x, _ = la.bicgstab(D,p)
print(x.reshape(bx,by))

64 35 64
(143360,)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 6

[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 

[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 

[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 

[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 

...

(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64

[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 

[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 

[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 35, 64)
[64 35 64]
(64, 

KeyboardInterrupt: 